# IBM Data Science - Professional Certificate
## Capstone Projects - Battle of Neighborhoods
### Author: Vitor Villas Boas

In [94]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [95]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
ACCESS_TOKEN = '' # your access token to user.id
RADIUS = 10000
LIMIT = 100

In [96]:
# !wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
# print('Data downloaded!')
CITY = "Sao Paulo, SP"
url_SP = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&v={}&radius={}&limit={}&near={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    ACCESS_TOKEN,
    VERSION,
    RADIUS,
    LIMIT,
    CITY)
results = requests.get(url_SP).json()

In [97]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [98]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) # flatten JSON
# nearby_venues.head()

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.head(10)

(100, 5)


,id,name,categories,lat,lng
0,4b17eb00f964a520a1c923e3,Centro Cultural Banco do Brasil (CCBB),Cultural Center,-23.547588,-46.634683
1,573dab3f498e4e0d56c67f4e,Casa de Francisca,Music Venue,-23.548733,-46.634763
2,4f29bb59e4b09b0140fba336,Teatro CCBB,Theater,-23.547644,-46.634733
3,4bdafa28c79cc928177a80e9,Theatro Municipal de São Paulo,Theater,-23.545477,-46.638812
4,4b09acf9f964a520701b23e3,Teatro Renault,Theater,-23.554120,-46.638695
5,4b05bc5ef964a520fee122e3,Mercado Municipal Paulistano,Market,-23.541673,-46.629742
6,560ee563498e447aba686327,A Casa do Porco,Brazilian Restaurant,-23.544887,-46.644622
7,54ac1385498e545686d23f88,Por um Punhado de Dólares,Coffee Shop,-23.548488,-46.645305
8,58f278f8e65d0c4b18f857f0,Kenzo Sushi,Sushi Restaurant,-23.557705,-46.636036
9,4b0588c7f964a520e8d922e3,Pinacoteca do Estado de São Paulo,Art Museum,-23.534735,-46.634001


In [99]:
QUERY = "Coffee"
url_SP = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&v={}&query={}&radius={}&limit={}&near={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    ACCESS_TOKEN,
    VERSION,
    QUERY,
    RADIUS,
    LIMIT,
    CITY)
Coffee_results = requests.get(url_SP).json()

In [100]:
coffee_venues = Coffee_results['response']['venues']
coffee_df = pd.json_normalize(coffee_venues)
coffee_df.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,location.crossStreet,venuePage.id
0,50afc640e4b0a2172d570cd6,Açaí Coffee,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1606235327,False,"Rua XV de Novembro, 240",-23.545237,-46.634642,"[{'label': 'display', 'lat': -23.5452373925255...",BR,São Paulo,SP,Brasil,"[Rua XV de Novembro, 240, São Paulo, SP]",NaN,NaN,NaN,NaN
1,5c45fb77dd8442002c6a4885,Modernista Coffee Stories,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1606235327,False,"R. Major Sertório, 92",-23.545687,-46.645006,"[{'label': 'display', 'lat': -23.5456867471013...",BR,São Paulo,SP,Brasil,"[R. Major Sertório, 92, São Paulo, SP, 01220-020]",01220-020,República,NaN,NaN
2,5a3971acacc5f50c9619d6f6,89°C Coffee Station,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1606235327,False,"Pç. da Liberdade, 169",-23.555484,-46.635238,"[{'label': 'display', 'lat': -23.5554842459156...",BR,São Paulo,SP,Brasil,"[Pç. da Liberdade, 169 (R. dos Estudantes), Sã...",01503-010,Liberdade,R. dos Estudantes,NaN
3,50b38da1e4b03641debdff27,Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606235327,False,Banco de Tókio - Mitsubishi,-23.563580,-46.653522,"[{'label': 'display', 'lat': -23.5635799770846...",BR,São Paulo,SP,Brasil,"[Banco de Tókio - Mitsubishi, São Paulo, SP]",NaN,NaN,NaN,NaN
4,59d509bd50a6f07a61452229,Enjoy Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606235327,False,"Rua São Bento, 377",-23.545709,-46.635745,"[{'label': 'display', 'lat': -23.5457092041654...",BR,São Paulo,SP,Brasil,"[Rua São Bento, 377, São Paulo, SP, 01011-100]",01011-100,Sé,NaN,NaN
5,4da0d5eec6e96ea8c9c3b35d,Coffee Lab,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1606235327,False,"R. Fradique Coutinho, 1340",-23.556060,-46.691380,"[{'label': 'display', 'lat': -23.5560595130229...",BR,São Paulo,SP,Brasil,"[R. Fradique Coutinho, 1340, São Paulo, SP, 05...",05416-001,NaN,NaN,NaN
6,4e946b1346900e559d42f445,Coffee Shop - Sodexo,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606235327,False,Praça Ramos- Votorantim Metais - 4º Andar,-23.545245,-46.638237,"[{'label': 'display', 'lat': -23.5452451218314...",BR,São Paulo,SP,Brasil,"[Praça Ramos- Votorantim Metais - 4º Andar, Sã...",NaN,NaN,NaN,NaN
7,5c7ab9543fcee8002c046ffe,Coffee Sweet Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1606235327,False,"R. Barão de Tefé, 168",-23.527303,-46.676149,"[{'label': 'display', 'lat': -23.527303, 'lng'...",BR,São Paulo,SP,Brasil,"[R. Barão de Tefé, 168, São Paulo, SP, 05003-040]",05003-040,NaN,NaN,NaN
8,517f9fc0e4b0a270f64603e2,Açai Coffee,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1606235327,False,NaN,-23.547012,-46.634300,"[{'label': 'display', 'lat': -23.547012, 'lng'...",BR,NaN,NaN,Brasil,NaN,NaN,NaN,NaN,NaN
9,4bd074b4a8b3a5938dde635f,Coffee Corner,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1606235327,False,"R. da Consolação, 2593",-23.556665,-46.663851,"[{'label': 'display', 'lat': -23.5566649483494...",BR,São Paulo,SP,Brasil,"[R. da Consolação, 2593 (Av. Rebouças), São Pa...",01416-001,NaN,Av. Rebouças,NaN


In [101]:
filtered_columns = ['name', 'categories', 'location.neighborhood', 'location.lat', 'location.lng', 'id'] # + [col for col in coffee_df.columns if col.startswith('location.')]
filtered_coffee = coffee_df.loc[:, filtered_columns]
# filtered_coffee.head()

filtered_coffee['categories'] = filtered_coffee.apply(get_category_type, axis=1)
filtered_coffee.columns = [column.split('.')[-1] for column in filtered_coffee.columns]
print(filtered_coffee.shape)
filtered_coffee.head()

(50, 6)


,name,categories,neighborhood,lat,lng,id
0,Açaí Coffee,Deli / Bodega,NaN,-23.545237,-46.634642,50afc640e4b0a2172d570cd6
1,Modernista Coffee Stories,Coffee Shop,República,-23.545687,-46.645006,5c45fb77dd8442002c6a4885
2,89°C Coffee Station,Coffee Shop,Liberdade,-23.555484,-46.635238,5a3971acacc5f50c9619d6f6
3,Coffee,Café,NaN,-23.563580,-46.653522,50b38da1e4b03641debdff27
4,Enjoy Coffee,Café,Sé,-23.545709,-46.635745,59d509bd50a6f07a61452229


In [103]:
address = 'Sao Paulo, SP'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-23.5506507 -46.6333824


In [129]:
venues_map = folium.Map(location=santander, zoom_start=15)

santander = [-23.551416753127207, -46.63375810307533]
folium.Marker(
    santander,
    popup='<b>Santander<b>',
    icon=folium.Icon(color='red',
                         prefix='fa',
                         icon='university')
).add_to(venues_map)

for lat, lng, label in zip(filtered_coffee.lat, filtered_coffee.lng, filtered_coffee.categories):
    folium.Marker(
        [lat, lng],
        popup=label,
        # size=10,
        icon=folium.Icon(color='beige',
                         icon_color ='black',
                         prefix='fa',
                         icon='coffee') # Coffee Icon HTML: <i class="fas fa-coffee"></i>
    ).add_to(venues_map)

# display map
venues_map

In [117]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `prefix` as well.
 |  angle : int, default 0
 |      The icon will be rotated by this amount of degrees.
 

In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    nyc_data = json.load(json_data)

In [ ]:
neighborhoods_data = nyc_data['features']
neighborhoods_data[0]

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)